In [1]:
import sys
sys.path.append("../scr/utils")


from copy import deepcopy
import time
import platform
from tqdm import tqdm

import torch
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader

from create_dataset import BirdDataset
from base_utils import set_seed
from metrics import validation_epoch_end

In [2]:
# Информация о железе, на котором тестируется модель

processor_info = platform.processor()
print("Информация о процессоре:", processor_info)

if torch.cuda.is_available():
    num_gpus = torch.cuda.device_count()
    print("Доступно GPU:", num_gpus)
    for i in range(num_gpus):
        gpu = torch.cuda.get_device_name(i)
        print(f"GPU {i + 1}: {gpu}")   
else:
    print("GPU недоступны на данной системе.")

Информация о процессоре: arm
GPU недоступны на данной системе.


In [3]:
df = pd.read_csv("../data/data.csv")
df_test = df[df.fold == 3].sample(n=100, random_state=42).reset_index(drop=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_base = torch.load("../experiment/14_September_2023_16_29/model_tf_efficientnet_b0_last_version.pt", map_location=device).to(device)
dataset_test = BirdDataset(df=df_test, path_to_folder_with_audio="../data")
valid_loader = DataLoader(dataset_test, batch_size=4)
model_base.eval()
device

device(type='cpu')

## Проверим работу базовой модели

In [4]:
model = deepcopy(model_base)

In [31]:
def get_model_size(model):
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()

    size_all_mb = (param_size + buffer_size) / 1024**2
    print('model size: {:.3f}MB'.format(size_all_mb))

In [32]:
def get_metric_score(model):
    predicted_labels_list = None
    true_labels_list = None
    metric = validation_epoch_end
    start_time = time.time()


    with torch.no_grad():
        for batch in tqdm(valid_loader):
            X_batch = batch[0].to(device)
            y_batch = batch[1].to(device)
            res = model.forward(X_batch)

            res = res.detach().sigmoid().cpu().numpy()
            y_batch_onehot = y_batch
            y_batch_onehot = y_batch_onehot.unsqueeze(1).detach().cpu().numpy()
            y_batch_onehot = y_batch_onehot.squeeze()

            if predicted_labels_list is None:
                predicted_labels_list = res
                true_labels_list = y_batch_onehot
            else:
                predicted_labels_list = np.concatenate([predicted_labels_list, res], axis=0)
                true_labels_list = np.concatenate([true_labels_list, y_batch_onehot], axis=0)

            del batch, res
        torch.cuda.empty_cache()

    end_time = time.time()


    all_predicted_labels = np.vstack(predicted_labels_list)
    all_true_labels = np.vstack(true_labels_list)
    all_true_labels = np.squeeze(all_true_labels)
    mask = (all_true_labels > 0) & (all_true_labels < 1)
    all_true_labels[mask] = 0
    avg_metric = metric(all_true_labels, all_predicted_labels)
    t = end_time - start_time

    # Мы понимаем, что ко времени работы модели добавляем время обработки батчей и добавления аугментаций к стартовым данным

    print("Наши метрики на нашей базовой модели:")
    print(f"Время работы модели на всем батче {t:.<2g} сек.")
    print(f"Время работы модели на одном сэмпле (AVG) {round(t/ len(dataset_test), 3) } сек.")
    print("Метрики качества:")
    for m in avg_metric:
        print(f"metric {m} : {avg_metric[m]:.<5g}")

In [27]:
get_model_size(model=model)

model size: 16.738MB


In [26]:
get_metric_score(model=model)

100%|██████████| 25/25 [00:22<00:00,  1.09it/s]

Наши метрики на нашей базовой модели:
Время работы модели на всем батче 22.8778 сек.
Время работы модели на одном сэмпле (AVG) 0.229 сек.
Метрики качества:
metric val_RMAP : 0.746389


## PTDQ  fp32 -> qint8

In [28]:
model = deepcopy(model_base)
torch.backends.quantized.engine = 'qnnpack'
model_int8 = torch.ao.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8  # the target dtype for quantized weights
)

In [35]:
get_model_size(model=model_int8)

model size: 15.448MB


In [36]:
get_metric_score(model=model_int8)

100%|██████████| 25/25 [00:24<00:00,  1.02it/s]

Наши метрики на нашей базовой модели:
Время работы модели на всем батче 24.4309 сек.
Время работы модели на одном сэмпле (AVG) 0.244 сек.
Метрики качества:
metric val_RMAP : 0.738694


#### Незначительное уменьшение веса модели и метрики

## PTDS  fp32 -> qint8

In [45]:
model = deepcopy(model_base)

In [46]:
model.eval();

In [70]:
model.qconfig = torch.ao.quantization.get_default_qconfig('x86')
# Fuse the activations to preceding layers, where applicable.
# This needs to be done manually depending on the model architecture.
# Common fusions include `conv + relu` and `conv + batchnorm + relu`
model_fp32_fused = torch.ao.quantization.fuse_modules(model, [[name for name, _ in model.named_modules() if name != '']])

# Prepare the model for static quantization. This inserts observers in
# the model that will observe activation tensors during calibration.
model_fp32_prepared = torch.ao.quantization.prepare(model_fp32_fused)

# calibrate the prepared model to determine quantization parameters for activations
# in a real world setting, the calibration would be done with a representative dataset
input_fp32 = torch.randn(4, 1, 4, 4)
model_fp32_prepared(input_fp32)

# Convert the observed model to a quantized model. This does several things:
# quantizes the weights, computes and stores the scale and bias value to be
# used with each activation tensor, and replaces key operators with quantized
# implementations.
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

# run the model, relevant calculations will happen in int8
res = model_int8(input_fp32);

AssertionError: did not find fuser method for: (<class 'timm.layers.conv2d_same.Conv2dSame'>, <class 'timm.layers.norm_act.BatchNormAct2d'>, <class 'torch.nn.modules.linear.Identity'>, <class 'torch.nn.modules.activation.SiLU'>, <class 'torch.nn.modules.container.Sequential'>, <class 'torch.nn.modules.container.Sequential'>, <class 'timm.models._efficientnet_blocks.DepthwiseSeparableConv'>, <class 'torch.nn.modules.conv.Conv2d'>, <class 'timm.layers.norm_act.BatchNormAct2d'>, <class 'torch.nn.modules.linear.Identity'>, <class 'torch.nn.modules.activation.SiLU'>, <class 'timm.models._efficientnet_blocks.SqueezeExcite'>, <class 'torch.nn.modules.conv.Conv2d'>, <class 'torch.nn.modules.activation.SiLU'>, <class 'torch.nn.modules.conv.Conv2d'>, <class 'torch.nn.modules.activation.Sigmoid'>, <class 'torch.nn.modules.conv.Conv2d'>, <class 'timm.layers.norm_act.BatchNormAct2d'>, <class 'torch.nn.modules.linear.Identity'>, <class 'torch.nn.modules.linear.Identity'>, <class 'torch.nn.modules.li

In [ ]:
get_model_size(model=model_int8)

In [ ]:
get_metric_score(model=model_int8)